参考 https://www.kesci.com/home/project/5a5e0af01badff1e49557e47

# 第11章 声呐返回值分类

## 导入数据

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data',header=None)

长知识了，第一次看到用网址作为数据url进行读取的。

In [2]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [3]:
data.shape

(208, 61)

In [4]:
data.iloc[:,60].value_counts() #所以这是个二分类问题

M    111
R     97
Name: 60, dtype: int64

可以看到数据最后一列是其所属的分类。所有的数据都是连续的（没有缺失值），从0~1，M代表矿石，R代表石头（这是想要挖矿？探测矿藏？）

## 创建模型

In [5]:
import numpy as np
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import  np_utils
from keras.models import Sequential

from sklearn.cross_validation import StratifiedKFold,cross_val_score,KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.
D:\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
seed=7
np.random.seed(seed)

In [7]:
dataset=data.values

In [8]:
X=dataset[:,0:60].astype('float')
Y=dataset[:,60]

In [9]:
encoder=LabelEncoder()
encoded_Y=encoder.fit_transform(Y)
dummy_Y=np_utils.to_categorical(encoded_Y)

In [10]:
encoded_Y[:4]  #因为是二分类，所以没使用独热码，其实不使用也可以，我的小论文也是二分类，我使用了独热码，可以看看使用和不使用的效果

array([1, 1, 1, 1], dtype=int64)

In [11]:
dummy_Y[:4]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [12]:
def create_baseline():
    model=Sequential()
    model.add(Dense(60,input_dim=60,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [13]:
estimator=KerasClassifier(build_fn=create_baseline,epochs=100,batch_size=5,verbose=0)

In [14]:
EY=encoded_Y
kfold=StratifiedKFold(y=EY,n_folds=10,shuffle=True,random_state=seed)
results=cross_val_score(estimator,X,EY,cv=kfold)
print('mean:%.2f  std:%.2f'%(results.mean(),results.std()))

mean:0.80  std:0.07


    EY=dummy_Y
    # kfold=StratifiedKFold(y=EY,n_folds=10,shuffle=True,random_state=seed)
    kfold=KFold(n=len(X),n_folds=10,shuffle=True,random_state=seed)
    results=cross_val_score(estimator,X,EY,cv=kfold)
    print('mean:%.2f  std:%.2f'%(results.mean(),results.std()))

>**因为构建网络时，最后一层只有一个输出，所以不同于鸢尾花，使用独热码，模型最后的输出也是3维。<br>
你的数据维度要与模型构建保持一致。如果这里模型最后一层是2个输出，那么就需要使用独热码将Y变成二维**

## 改进模型

### 数据预处理

+ 因为这里每一行数据（每一条记录）都有60中特征，为了保证每列分布比较均匀（在进行机器学习模型训练时，最好保证数据的分布一致），使用标准化，让每个数据列的均值为0，方差为0。
+ 注意，不能在整个数据集上直接应用标准化，应该只在测试数据进行交叉验证时进行标准化处理，使得标准化成为交叉验证的一环，让模型没有新数据的先验知识，防止模型发散。。。（这几句翻译，其实我觉得并不合理）

    steps : list
        List of (name, transform) tuples (implementing fit/transform) that are
        chained, in the order in which they are chained, with the last object
        an estimator.
 
可放在Pipeline中的步骤可能有：

+ 特征标准化是需要的，可作为第一个环节
+ 既然是分类器，classifier也是少不了的，自然是最后一个环节
+ 中间可加上比如数据降维（PCA）
+ ……

In [15]:
pipeline=Pipeline([('Standardize',StandardScaler()),
                  ('mlp',KerasClassifier(build_fn=create_baseline,epochs=100,batch_size=5,verbose=0)),    
                  ])
kfold=StratifiedKFold(y=encoded_Y,n_folds=10,shuffle=True,random_state=seed)
results=cross_val_score(pipeline,X,encoded_Y,cv=kfold)

In [16]:
print('mean:%.2f%%  std:%.2f%%'%(results.mean()*100,results.std()*100))

mean:84.59%  std:5.20%


没有标准化的是 mean:0.82  std:0.08，相比之下，标准化之后提升了还不少。每次运行得到的结果都不一样。。。

### 调整模型拓扑和神经元

#### 缩小网络

In [17]:
def create_baseline():
    model=Sequential()
    model.add(Dense(30,input_dim=60,kernel_initializer='normal',activation='relu')) 
    #在这里试试特征提取，把输入的60维度提取成为30d
    model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

pipeline=Pipeline([('Standardize',StandardScaler()),
                  ('mlp',KerasClassifier(build_fn=create_baseline,epochs=100,batch_size=5,verbose=0)),    
                  ])
kfold=StratifiedKFold(y=encoded_Y,n_folds=10,shuffle=True,random_state=seed)
results=cross_val_score(pipeline,X,encoded_Y,cv=kfold)

In [18]:
print('mean:%.2f%%  std:%.2f%%'%(results.mean()*100,results.std()*100))

mean:84.61%  std:5.12%


缩小网络降低的不是很多，还可以

#### 扩大网络

def create_baseline():
    model=Sequential()
    model.add(Dense(60,input_dim=60,kernel_initializer='normal',activation='relu')) 
    model.add(Dense(30,kernel_initializer='normal',activation='relu')) #多加一个隐藏层，帮助平稳过渡删选特征
    model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

pipeline=Pipeline([('Standardize',StandardScaler()),
                  ('mlp',KerasClassifier(build_fn=create_baseline,epochs=100,batch_size=5,verbose=0)),    
                  ])
kfold=StratifiedKFold(y=encoded_Y,n_folds=10,shuffle=True,random_state=seed)
results=cross_val_score(pipeline,X,encoded_Y,cv=kfold)

In [20]:
print('mean:%.2f%%  std:%.2f%%'%(results.mean()*100,results.std()*100))

mean:85.07%  std:4.54%


准确率均值没有很大变化，但是这个网络的std更小，说明这个网络鲁棒性更好了

# 第16章 使用dropout防止过拟合

先把上面代码全部运行一遍，这样就不用再导入重复的库或者数据了

与11章相比，16章中compile中optimizer修改成了SGD

## 不使用dropout

In [21]:
from keras.optimizers import SGD

In [28]:
def create_baseline():
    model=Sequential()
    model.add(Dense(60,input_dim=60,kernel_initializer='normal',activation='relu')) 
    model.add(Dense(30,kernel_initializer='normal',activation='relu')) #多加一个隐藏层，帮助平稳过渡删选特征
    model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
    
    sgd=SGD(lr=0.01,momentum=0.8,decay=0.0,nesterov=False)
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

pipeline=Pipeline([('Standardize',StandardScaler()),
                  ('mlp',KerasClassifier(build_fn=create_baseline,epochs=300,batch_size=16,verbose=0)),    
                  ])
kfold=StratifiedKFold(y=encoded_Y,n_folds=10,shuffle=True,random_state=seed)
results=cross_val_score(pipeline,X,encoded_Y,cv=kfold)

In [29]:
print('mean:%.2f%%  std:%.2f%%'%(results.mean()*100,results.std()*100))

mean:85.54%  std:4.88%


## 使用dropout

https://blog.csdn.net/program_developer/article/details/80737724

Dropout可以比较有效的缓解过拟合的发生，在一定程度上达到正则化的效果。

Dropout可以作为训练深度神经网络的一种trick供选择。在每个训练批次中，通过忽略一半的特征检测器（让一半的隐层节点值为0），可以明显地减少过拟合现象。这种方式可以减少特征检测器（隐层节点）间的相互作用，检测器相互作用是指某些检测器依赖其他检测器才能发挥作用。

Dropout说的简单一点就是：我们在前向传播的时候，让某个神经元的激活值以一定的概率p停止工作，这样可以使模型泛化性更强，因为它不会太依赖某些局部的特征。
![image](https://img-blog.csdn.net/20180619185225799?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3Byb2dyYW1fZGV2ZWxvcGVy/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

去掉神经元，和文中说的，每五个输入丢掉一个，在实际效果上这样说没错，但是但从含义上来说，是不对的。

https://keras.io/layers/core/ 在这个页面搜索dropout 这里的输入不是指狭义上的输入层，是指对每层的输入进行一种小处理

### 对输入层使用dropout

In [26]:
from keras.layers import Dropout
from keras.constraints import maxnorm

原论文推荐：

对每层的权重加限制，保证模不超过3：，在定义全连接层时使用w_constraint可以做到。学习率加10倍，动量加到0.9

https://keras.io/constraints/ 原文代码中使用的w_constraint已经使用kernel_constraint替代了

In [31]:
def create_baseline():
    model=Sequential()
    model.add(Dropout(0.2,input_shape=(60,)))
    model.add(Dense(60,kernel_initializer='normal',activation='relu',kernel_constraint=maxnorm(3))) 
    model.add(Dense(30,kernel_initializer='normal',activation='relu',kernel_constraint=maxnorm(3))) #多加一个隐藏层，帮助平稳过渡删选特征
    model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
    
    sgd=SGD(lr=0.1,momentum=0.9,decay=0.0,nesterov=False)
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

pipeline=Pipeline([('Standardize',StandardScaler()),
                  ('mlp',KerasClassifier(build_fn=create_baseline,epochs=100,batch_size=5,verbose=0)),    
                  ])
kfold=StratifiedKFold(y=encoded_Y,n_folds=10,shuffle=True,random_state=seed)
results=cross_val_score(pipeline,X,encoded_Y,cv=kfold)

In [32]:
print('mean:%.2f%%  std:%.2f%%'%(results.mean()*100,results.std()*100))

mean:87.95%  std:2.54%


比不加的 mean:85.54%  std:4.88% 高了

### 对隐层使用dropout

In [33]:
def create_baseline():
    model=Sequential()
    model.add(Dense(60,input_dim=60,kernel_initializer='normal',activation='relu',kernel_constraint=maxnorm(3))) 
    model.add(Dropout(0.2))
    model.add(Dense(30,kernel_initializer='normal',activation='relu',kernel_constraint=maxnorm(3))) 
    model.add(Dropout(0.2))
    model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
    
    sgd=SGD(lr=0.1,momentum=0.9,decay=0.0,nesterov=False)
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

pipeline=Pipeline([('Standardize',StandardScaler()),
                  ('mlp',KerasClassifier(build_fn=create_baseline,epochs=100,batch_size=5,verbose=0)),    
                  ])
kfold=StratifiedKFold(y=encoded_Y,n_folds=10,shuffle=True,random_state=seed)
results=cross_val_score(pipeline,X,encoded_Y,cv=kfold)

In [34]:
print('mean:%.2f%%  std:%.2f%%'%(results.mean()*100,results.std()*100))

mean:85.50%  std:5.49%


论调参的重要性